In [2]:
import torch
import numpy as np
from numpy.linalg import norm
from sklearn.cluster import KMeans, DBSCAN
import torch.nn.functional as F

In [3]:
def get_confusion_matrix(idx2category, labels_all, logits_all):
    # modelnet_dict = torch.load("src/eval_data/modelnet_dict.pt")

    cat_list = list(idx2category.values())
    
    print("labels_all: ", labels_all.shape)
    
    _, pred = logits_all.topk(1, 1, True, True)
    pred = pred.reshape(-1)
    
    true_pred = torch.argwhere(pred == labels_all).reshape(-1)
    false_pred = torch.argwhere(pred != labels_all).reshape(-1)
    
    res = torch.zeros(len(cat_list), len(cat_list))
    print("res: ", res.shape)
    for val in true_pred:
        idx = val.item()
        res[labels_all[idx], pred[idx]] += 1
    
    for val in false_pred:
        idx = val.item()
        res[labels_all[idx], pred[idx]] += 1
        # print(labels_all[idx], pred[idx])

    
    res_norm = F.softmax(res, dim=1)

    return res, res_norm

In [11]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
clip_feat_path = "../meta_data/lvis_cat_name_pt_feat.npy"
clip_cat_feat = np.load(clip_feat_path, allow_pickle=True)
objaverse_dict = torch.load("../src/eval_data/objaverse_dict.pt")
ov_category2idx = objaverse_dict["category2idx"]
ov_idx2category = objaverse_dict["idx2category"]
ov_category2idx = objaverse_dict["category2idx"]

ov_labels_all = objaverse_dict["labels_all"]
ov_logits_all = objaverse_dict["logits_all"]

objaverse_dict = None
# print("clip_cat_feat: ", clip_cat_feat.shape) # (1156, 1280)
# clip_cat_feat = F.normalize(clip_cat_feat, dim=-1)
clip_cat_feat = clip_cat_feat / norm(clip_cat_feat, axis=1, keepdims=True)

eps = 0.61855
db_scan = DBSCAN(eps=eps, min_samples=2).fit(clip_cat_feat)
dbscan_labels = db_scan.labels_

In [12]:
unique_labels = np.unique(dbscan_labels)
cluster_dict = {}
for l in unique_labels:
    if l == -1:
        continue
    idx = np.argwhere(dbscan_labels == l).reshape(-1)
    cluster_dict[l] = idx.tolist()


In [13]:
for k, v in cluster_dict.items():
    if len(v) > 7:
        print(f"Cluster {k}: {len(v)}")

Cluster 1: 8
Cluster 4: 9
Cluster 6: 28
Cluster 12: 18
Cluster 14: 9
Cluster 15: 26
Cluster 24: 23
Cluster 27: 14
Cluster 29: 9
Cluster 41: 21
Cluster 43: 23
Cluster 46: 10
Cluster 55: 8
Cluster 56: 12
Cluster 59: 8
Cluster 82: 9
Cluster 106: 8
Cluster 128: 9


In [14]:
## Lets analyze Cluster 56: 
c_no = 56

In [15]:
cluster_dict[c_no]

[143, 248, 367, 480, 496, 769, 796, 867, 902, 906, 986, 1057]

In [16]:
# baseline_labels_all = torch.load("../exp/finetune_1layer@20240129-151703/results/ov_labels_all_finetune_test.pt")
# baseline_logits_all = torch.load("../exp/finetune_1layer@20240129-151703/results/ov_logits_all_finetune_test.pt")
confusion_mat, confusion_mat_norm = get_confusion_matrix(ov_idx2category, ov_labels_all, ov_logits_all)

labels_all:  torch.Size([46205])
res:  torch.Size([1156, 1156])


In [17]:
confusion_mat[cluster_dict[c_no]].shape

torch.Size([12, 1156])

In [62]:
cluster_dict[c_no][:5]

[143, 248, 367, 480, 496]

In [54]:
torch.sum(a, dim=1)

tensor([-1.8075,  3.8043,  2.8475])

In [60]:
def top_confusion(ov_idx2category, ov_confusion, ov_res_norm, dim=1, topx=40, topy=5):

    # mn_res_norm = torch.load("src/eval_data/mn_confusion_norm.pt")
    # ov_res_norm = torch.load("eval_data/ov_confusion_norm.pt")

    ov_res_norm = ov_res_norm.fill_diagonal_(0)
    row_sum = torch.sum(ov_res_norm, dim=dim)
    # print("row_sum: ", row_sum.shape) # 1156 categories

    sorted_idx = torch.argsort(row_sum, descending=True)

    # print("row_sum: ", row_sum[sorted_idx[:200]] )
    print("sorted: ", sorted_idx.shape)
    sort_cat = [ov_idx2category[i.item()] for i in sorted_idx[:topx]]

    # print("sorted: ", sorted_idx[:topx], '\n')
    top_xy_dict = {}

    for i in sorted_idx[:topx]:
        topy_cat=[]
        # sim_list = []
        confusion = []
        # top_xy_dict[i.item()] = {"cat_name":[], "count": [], "cats": []}
        topy_pred = torch.argsort(ov_res_norm[i], descending=True)[:topy]
        for cat_i in topy_pred:
            topy_cat.append(ov_idx2category[cat_i.item()])
            # sim_list.append(sim_mat[i][cat_i])
            confusion.append(ov_confusion[i][cat_i].item())
        
        # top_xy_dict[i.item()]["cats"] = topy_pred.tolist()
        # top_xy_dict[i.item()]["cat_name"] = topy_cat
        # top_xy_dict[i.item()]["count"] = confusion

        # topy_cat = [ov_idx2category[i.item()] for i in topy_pred]
        print(ov_idx2category[i.item()], ": ", topy_cat)
        # print(ov_idx2category[i.item()], ": ", sim_list)
        print(ov_idx2category[i.item()], ": ", confusion, '\n')
        # print(ov_idx2category[i.item()], ": ", ov_res_norm[i][topy_pred])
    
    # print("top_xy_dict: ", top_xy_dict)
    # np.save("src/eval_data/top_xy_dict.npy", top_xy_dict)


In [56]:
filter_confusion = confusion_mat[cluster_dict[c_no]]
filter_confusion_norm = confusion_mat_norm[cluster_dict[c_no]]
filter_confusion_norm.shape

torch.Size([12, 1156])

In [66]:
cat_idx_from_cluster = cluster_dict[c_no][:5]
filter_confusion = confusion_mat[cat_idx_from_cluster]
filter_confusion_norm = confusion_mat_norm[cat_idx_from_cluster]
top_confusion(ov_idx2category, filter_confusion, filter_confusion_norm, dim=1, topx=12, topy=5)

sorted:  torch.Size([5])
Band_Aid :  ['briefcase', 'suitcase', 'shoulder_bag', 'handbag', 'shopping_bag']
Band_Aid :  [18.0, 8.0, 2.0, 2.0, 1.0] 

Dixie_cup :  ['handbag', 'shoulder_bag', 'tote_bag', 'clutch_bag', 'duffel_bag']
Dixie_cup :  [14.0, 13.0, 11.0, 3.0, 1.0] 

CD_player :  ['backpack', 'duffel_bag', 'shoulder_bag', 'handbag', 'satchel']
CD_player :  [10.0, 7.0, 4.0, 4.0, 1.0] 

Christmas_tree :  ['shopping_bag', 'tote_bag', 'grocery_bag', 'packet', 'cube']
Christmas_tree :  [6.0, 5.0, 3.0, 2.0, 1.0] 

Bible :  ['backpack', 'shoulder_bag', 'satchel', 'suitcase', 'handbag']
Bible :  [4.0, 4.0, 2.0, 2.0, 2.0] 



In [64]:
cluster_dict[c_no]
[ov_idx2category[i] for i in cluster_dict[c_no]]

['briefcase',
 'clutch_bag',
 'duffel_bag',
 'grocery_bag',
 'handbag',
 'plastic_bag',
 'pouch',
 'satchel',
 'shopping_bag',
 'shoulder_bag',
 'suitcase',
 'tote_bag']

In [69]:
print(ov_category2idx["briefcase"])
print(ov_category2idx["Band_Aid"])

143
0


In [72]:
confusion_mat[143][143]

tensor(18.)

In [49]:
for idx, val in enumerate(confusion_mat[986]):
    if val > 0:
        print(f"idx: {idx}, val: {val}")
        # print(val.item())


idx: 42, val: 1.0
idx: 143, val: 8.0
idx: 189, val: 2.0
idx: 230, val: 2.0
idx: 311, val: 1.0
idx: 367, val: 1.0
idx: 503, val: 1.0
idx: 609, val: 1.0
idx: 867, val: 1.0
idx: 906, val: 2.0
idx: 986, val: 24.0
idx: 1077, val: 1.0


In [28]:
filter_confusion = confusion_mat[cluster_dict[c_no]]
filter_confusion_norm = confusion_mat_norm[cluster_dict[c_no]]
top_confusion(ov_idx2category, filter_confusion, filter_confusion_norm, dim=1, topx=40, topy=5)

Sharpie :  ['tote_bag', 'shoulder_bag', 'basket', 'handbag', 'clutch_bag']
Sharpie :  [22.0, 7.0, 4.0, 4.0, 2.0] 

Rollerblade :  ['shoulder_bag', 'backpack', 'clutch_bag', 'handbag', 'ski_boot']
Rollerblade :  [11.0, 4.0, 4.0, 2.0, 1.0] 

Ferris_wheel :  ['pouch', 'shopping_bag', 'beanbag', 'chime', 'backpack']
Ferris_wheel :  [3.0, 2.0, 2.0, 1.0, 1.0] 

Bible :  ['backpack', 'shoulder_bag', 'satchel', 'suitcase', 'handbag']
Bible :  [4.0, 4.0, 2.0, 2.0, 2.0] 

Lego :  ['backpack', 'pottery', 'clutch_bag', 'vase', 'wall_socket']
Lego :  [4.0, 3.0, 2.0, 2.0, 1.0] 

Christmas_tree :  ['shopping_bag', 'tote_bag', 'packet', 'shoulder_bag', 'cube']
Christmas_tree :  [7.0, 4.0, 3.0, 2.0, 1.0] 

CD_player :  ['backpack', 'shoulder_bag', 'handbag', 'beanbag', 'tote_bag']
CD_player :  [9.0, 4.0, 3.0, 1.0, 1.0] 

Dixie_cup :  ['shoulder_bag', 'tote_bag', 'clutch_bag', 'hatbox', 'headscarf']
Dixie_cup :  [15.0, 9.0, 4.0, 1.0, 1.0] 

air_conditioner :  ['shoulder_bag', 'packet', 'bucket', 'duffel

In [13]:
for k, v in cluster_dict.items():
    print(f"Cluster {k}: ")
    print([ov_idx2category[i] for i in v])

Cluster 0: 
['Band_Aid', 'bandage']
Cluster 1: 
['Bible', 'book', 'booklet', 'diary', 'hardback_book', 'notebook', 'notepad', 'paperback_book']
Cluster 2: 
['CD_player', 'phonograph_record', 'record_player']
Cluster 3: 
['Ferris_wheel', 'steering_wheel', 'wagon_wheel', 'wheel']
Cluster 4: 
['Rollerblade', 'ice_skate', 'roller_skate', 'skateboard', 'ski', 'ski_boot', 'ski_pole', 'snowboard', 'water_ski']
Cluster 5: 
['Tabasco_sauce', 'hot_sauce']
Cluster 6: 
['aerosol_can', 'barrel', 'beer_bottle', 'beer_can', 'bottle', 'can', 'canister', 'chinaware', 'cooking_utensil', 'cream_pitcher', 'cylinder', 'flowerpot', 'frying_pan', 'jar', 'keg', 'pan_(for_cooking)', 'pan_(metal_container)', 'pitcher_(vessel_for_liquid)', 'pot', 'pottery', 'saucepan', 'thermos_bottle', 'urn', 'vase', 'water_bottle', 'water_jug', 'watering_can', 'wine_bottle']
Cluster 7: 
['airplane', 'fighter_jet', 'helicopter', 'jet_plane', 'seaplane']
Cluster 8: 
['alarm_clock', 'clock', 'clock_tower', 'pocket_watch', 'timer'

In [3]:
obj_idx = ov_category2idx["motor_vehicle"]
val = dbscan_labels[obj_idx]
idx = np.argwhere(dbscan_labels == val)

obj_list = []
for i in idx:
    obj_list.append(ov_idx2category[i.item()])

print("obj_list: ", len(obj_list))
print("obj_list: ", obj_list)


obj_list:  6
obj_list:  ['car_(automobile)', 'convertible_(automobile)', 'generator', 'motor', 'motor_vehicle', 'race_car']


In [8]:
top_xy_dict = np.load("src/eval_data/top_xy_dict.npy", allow_pickle=True).item()
# def get_cat_count(top_xy_dict):
#     cat = top_xy_dict["cats"]
#     count = top_xy_dict["count"]


In [5]:
# top_xy_dict

In [9]:
def get_cluster(idx, dbscan_labels):
    val = dbscan_labels[idx]
    idx = np.argwhere(dbscan_labels == val)

    obj_list = []
    for i in idx:
        obj_list.append(ov_idx2category[i.item()])

    print("obj_list: ", len(obj_list))
    print("obj_list: ", obj_list)

In [7]:
top_40 = [1101,  142,  114,  169,  898,  426, 1025,  312,  438,  950,  758,  554,
         826, 1083,  118, 1037,  122,  612,  386,  902,  189,  568,  270,  903,
         464,  618,  278,  372,  724,  619,  658,  414,  667,  210,  229,  410,
         678,  483,  106, 1019]

In [10]:
cat_idx = 1019
print(ov_idx2category[cat_idx])

teakettle


In [11]:
get_cluster(cat_idx, dbscan_labels)

obj_list:  9
obj_list:  ['coffee_maker', 'coffeepot', 'cup', 'kettle', 'mug', 'teacup', 'teakettle', 'teapot', 'trophy_cup']


In [12]:
top_xy_dict[cat_idx]

{'cat_name': ['teapot',
  'kettle',
  'plume',
  'pocketknife',
  'plow_(farm_equipment)'],
 'count': [59.0, 2.0, 0.0, 0.0, 0.0],
 'cats': [1020, 562, 775, 777, 774]}

In [ ]:
[464, 409, 861]
[372, 510]
[106, 548]
[1019, 1020, 562]

1019, 106, 372, 464

In [ ]:
[464, 409, 861]
[372, 510]
[106, 548]
[1019, 1020, 562]

arr = [464, 409, 861, 372, 510, 106, 548, 1019, 1020, 562]

In [ ]:
# print("dbscan: ", db_scan.labels_)
print("eps: ", eps)
print("dbscan: ", np.unique(db_scan.labels_))